# Processing the videos

## import the basic package

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
import visualize

import cv2


## Get Directory

In [ ]:
%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your 
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configuration

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

## Create model and load trained weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights('./mask_rcnn_coco.h5', by_name=True)

## Class Name

In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Open Video and Run Object Detection

In [ ]:
video_path = './videos'
video_names = next(os.walk(video_path))[2]

index = 0
for video_name in video_names:
    file_name = video_name
    video_name = os.path.join(video_path, video_name)
    vc = cv2.VideoCapture(video_name)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    vw = cv2.VideoWriter(video_name+'rest.avi', fourcc, 20.0, (320, 240))
    
    if not vc.isOpened():
        print('cannot open', video_name)
    print('handling ', video_name)
    ret, frame = vc.read()
    while ret:
        # translate nparray to sci-image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model.detect([frame], verbose=0)
        r =  r = results[0]
        result_frame = visualize.get_instances_image(frame, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])
        vw.write(result_frame)
        cv2.imwrite(file_name+'_'+str(index)+'.png', result_frame)
        index += 1
        ret, frame = vc.read()